# Игра в крестики-нолики

Для того чтобы запустить игру нужно загрузить все классы и код с игрой (выполнить все ячейки с кодом)

## Core классы игры

In [53]:
class Game:
    
    def __init__(self, field=None):
        self.field = None
        
        if field:
            self.field = field
        else:
            self.start()
    
    def start(self):
        self.field = [' ']*9

    def printField(self):
        row = ''
        for i in range(len(self.field)):
            cell = self.field[i]
            row+='['
            if cell != ' ':
                row+=cell
            else:
                row+=str(i+1)
            row+=']'
            if (i % 3 == 2): 
                print(row)
                row = ''
    
    def set(self, position, side):
        pos = int(position)-1
        self.field[pos] = side

    def getFree(self):
        free = []
        for i in range(len(self.field)):
            cell = self.field[i]
            if (cell == ' '):
                free.append( (i+1) )
        return free
    
    def isDraw(self):
        free = self.getFree();
        return len(free)==0;

    def isWin(self, side):
        for i in range(3):
            isW = True 
            for j in range(3):
                if self.field[i*3+j]!=side:
                    isW = False
                    break
            if isW:
                return isW
            
        for i in range(3):
            isW = True 
            for j in range(3):
                if self.field[j*3+i]!=side:
                    isW = False
                    break
            if isW:
                return isW
            
        isW = True;
        for i in range(3):
            if self.field[i*3+i]!=side:
                isW = False
                break
        if isW:
            return isW
        
        isWi = True;
        for i in range(3):
            if self.field[(i*3+2-i)]!=side:
                isW = False
                break
        if isW:
            return isW

        return False

    def getState(self, side):
        if side == 'x':
            return self.field

        newField = ''
        for i in range(len(self.field)):
            if self.field[i]=='x':
                newField += 'o'
            elif self.field[i]=='o':
                newField += 'x'
            else:
                newField += self.field[i] 

        return newField

In [54]:
import os
import json

class AI:
    def __init__(self):
        self.table = {}
        if os.path.isfile('./rewards.json'):
            with open('rewards.json') as json_file:
                self.table = json.load(json_file)
                print("loaded AI from rewards.json")

    def getReward(self, state):
        game = Game(state)
        
        #если победитель - мы, то оценка состояния игры "1"
        if game.isWin('x'):
            return 1

        #если победиль - соперник, то оценка состояния игры "0"
        if game.isWin('o'):
            return 0

        #смотрим ценность по таблице
        strstate = ''.join(state)
        if strstate in self.table.keys():
            return self.table[strstate]

        #если в таблице нет, то считаем начальной ценностью "0.5"
        return 0.5

    def correct(self, state, newReward):
        oldReward = self.getReward(state)
        strstate = ''.join(state)
        self.table[strstate] = oldReward + 0.1 * (newReward - oldReward)

    def save(self):
        with open('rewards.json', 'w') as outfile:
            json.dump(self.table, outfile)

In [55]:
import random 
import copy

class AIPlayer:
    
    def __init__(self, side, ai, isGreedy=True):
        self.side = side
        self.ai = ai
        self.oldState = None
        self.isGreedy = isGreedy

    def getSide(self):
        return self.side

    def makeStep(self, game):
        #получаем список доступных ходов
        free = game.getFree()
        
        #решаем, является ли текущий ход 
        #зондирующим (случайным) или жадным (максимально выгодным)
        
        if not self.isGreedy:
            #случайный ход
            print('Random step')
            step = random.choice(free)
            game.set(step, self.side)
            self.oldState = game.getState(self.side)
            return step

        #жадный ход
        rewards = {}
        for step in free:
            # для каждого доступного хода оцениваем состояние игры после него
            newGame = copy.deepcopy(game)
            newGame.set(step, self.side)
            rewards[step] = self.ai.getReward(newGame.getState(self.side))

        #выясняем, какое вознаграждение оказалось максимальным
        maxReward = 0
        for reward in rewards.values():
            if reward > maxReward:
                maxReward = reward

        #находим все шаги с максимальным вознаграждением
        steps = []
        
        for step in rewards:
            reward = rewards[step]
            if (maxReward > (reward - 0.01)) and (maxReward < (reward + 0.01)):
                steps.append(step)

        #корректируем оценку прошлого состояния
        #с учетом ценности нового состояния
        if (self.oldState):
            self.ai.correct(self.oldState, maxReward)

        #выбираем ход из ходов с максимальный вознаграждением
        step = random.choice(steps)
        game.set(step, self.side)

        #сохраняем текущее состояние для того, 
        #чтобы откорректировать её ценность на следующем ходе
        self.oldState = game.getState(self.side)
        return step

    def loose(self):
        #корректируем ценность предыдущего состояния при проигрыше
        if self.oldState:
            self.ai.correct(self.oldState, 0)

    def win(self):
        #корректируем ценность предыдущего состояния при выигрыше
        if self.oldState:
            self.ai.correct(self.oldState, 1)
    
    def draw(self):
        #корректируем ценность предыдущего состояния при ничьей
        if self.oldState:
            self.ai.correct(self.oldState, 0.5)

In [56]:
class UserPlayer:

    def __init__(self, side):
        self.side = side

    def getSide(self):
        return self.side

    def makeStep(self, game):
        game.printField()
        
        free = game.getFree()

        inp = None
        while (True):
            inp = input()
            if int(inp) in free:
                break
        
        game.set(inp, self.side)

    def loose(self):
        print('you loose')

    def win(self):
        print('you win')

    def draw(self):
        print('draw')

## Игра

In [57]:
print('Choose your side:')
print('X) x')
print('O) o')
print('Any other symbol if you would like to run AI vs AI')
side = input();

ai = AI()
gameCount = 1

playerX = None
playerO = None

if (side=='X') or (side=='x'):
    playerX = UserPlayer('x')
    playerO = AIPlayer('o', ai, True)
elif (side=='O') or (side=='o'):
    playerX = AIPlayer('x', ai, True)
    playerO = UserPlayer('o')
else:
    playerX = AIPlayer('x', ai, True)
    playerO = AIPlayer('o', ai, True)

    print('Enter games count:')
    gameCount = int(input())

    if (gameCount <= 0):
        gameCount = 1

game = Game()
for i in range(gameCount):
    print('New game', i+1)
    game.start()
    while (True):
        if game.isDraw():
            playerX.draw()
            playerO.draw()
            break

        playerX.makeStep(game)
        if game.isWin(playerX.getSide()):
            playerX.win()
            playerO.loose()
            break

        if game.isDraw():
            playerX.draw()
            playerO.draw()
            break

        field = playerO.makeStep(game)
        if game.isWin(playerO.getSide()):
            playerO.win()
            playerX.loose()
            break
    game.printField()

ai.save()

Choose your side:
X) x
O) o
Any other symbol if you would like to run AI vs AI

loaded AI from rewards.json
Enter games count:
2
New game 1
[o][o][x]
[x][x][o]
[x][o][x]
New game 2
[1][x][o]
[o][x][6]
[7][x][9]


## Жадный Агент

При запуске жадного агента нужно чтобы файл rewards.json находился в том же каталоге что и запускаемый код

In [58]:
def state_to_gmstate(state):
    """
    Переводим состояние игры из формата массива в формат строки,
    который используется игрой
    """
    gmstate = []
    for row in state:
        for ch in row:
            if ch == -1:
                gmstate.append(' ')
            elif ch == 1:
                gmstate.append('x')
            else:
                gmstate.append('o')
    return gmstate

In [59]:
def gmstep_to_step(step):
    """
    Переводим обозначение хода из принятого в игре 
    в формат который ожидается на выходе агента
    """
    steps = {
        1:(1,1),
        2:(1,2),
        3:(1,3),
        4:(2,1),
        5:(2,2),
        6:(2,3),
        7:(3,1),
        8:(3,2),
        9:(3,3),
    }
    return steps[step]

In [60]:
class AgentTicTacGreedy:
  def __init__(self, is_zero):
    #is_zero == True если нолик
    self.player = None
    ai = AI()
    if is_zero:
        self.player = AIPlayer('o', ai, True)
    else:
        self.player = AIPlayer('x', ai, True)

  def get_action(self, state):
    # пример state = [[1,0,1],[0,1,0],[0,1,0]] 1 - это крестик, 0 - это нолик, -1 - пусто
    gmstate = state_to_gmstate(state)
    game = Game(field=gmstate)
    move = self.player.makeStep(game)
    return gmstep_to_step(move) # возвращаем координаты хода

  def is_done(self, state, reward):
    # reward - вознаграждение 1 если выиграли , вызывается когда игра закончена
    gmstate = state_to_gmstate(state)
    game = Game(field=gmstate)
    return game.isWin(self.player.getSide())
    

## Рандомный Агент

In [61]:
class AgentTicTacRandom:
  def __init__(self, is_zero):
    #is_zero == True если нолик
    self.player = None
    ai = AI()
    if is_zero:
        self.player = AIPlayer('o', ai, False)
    else:
        self.player = AIPlayer('x', ai, False)

  def get_action(self, state):
    # пример state = [[1,0,1],[0,1,0],[0,1,0]] 1 - это крестик, 0 - это нолик, -1 - пусто
    gmstate = state_to_gmstate(state)
    game = Game(field=gmstate)
    move = self.player.makeStep(game)
    return gmstep_to_step(move) # возвращаем координаты хода

  def is_done(self, state, reward):
    # reward - вознаграждение 1 если выиграли , вызывается когда игра закончена
    gmstate = state_to_gmstate(state)
    game = Game(field=gmstate)
    return game.isWin(self.player.getSide())

### Проверяем

In [62]:
greedy, rand = AgentTicTacGreedy(False), AgentTicTacRandom(True)

loaded AI from rewards.json
loaded AI from rewards.json


In [63]:
greedy.is_done([[1,0,1],[0,1,0],[0,1,0]], None)

False

In [64]:
rand.is_done([[0,0,1],[0,1,0],[0,1,1]], None)

True

In [65]:
greedy.get_action([[1,-1,1],[0,0,0],[0,1,0]])

(1, 2)

In [66]:
rand.get_action([[1,-1,1],[0,-1,0],[0,1,0]])

Random step


(2, 2)